In [8]:
import bs4 as bs
import urllib.request
import pandas as pd
import re
import codecs
import os.path
import glob
import csv
import nltk
import numpy as np

# First We collect list of all movies and Filter it according to our Labels , After That Collect Full Review of Each Movie and create features from That.

In [2]:
#Read the page and create BeautifulSoup Object
link='https://wogma.com/movies/basic/'
source_page = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source_page,'lxml')

In [3]:
#From soup Object find the table tag
# If there is more than one table tag in Page use find_all('table')
table = soup.find('table')
#find all row tag in table
table_rows = table.find_all('tr')

In [4]:
#Iterate through each row and find all table data tag 


movie_data = []
for tr in table_rows:
    td = tr.find_all('td')
    row = []
    for col in td:
        #Iterate through all the td tags and if tag has class 'listing_synopsis' 
        # That Tag have link of the movie  
        #We extract link from <a> 
        if col.has_key('class') and col['class'][0] == 'listing_synopsis':
            hrefs = col.div.find('a')
            value = hrefs.get('href').strip()
        else:
            value = re.sub("[^a-zA-Z0-9’/_.']"," ",col.text).strip()
        row.append(value)
        # append 'Movie name' , 'Movie Rating' and 'Link of Full Review'
    movie_data.append(row)
    

C:\Users\Ali\Anaconda3\lib\site-packages\bs4\element.py:1577: UserWarning: has_key is deprecated. Use has_attr("class") instead.
  key))


In [5]:
#Total of 1253 moview data
len(movie_data)

1253

In [6]:
#We Wil Filter The data 
#We consider only those movie which has proper class defined by us and discard all remaining data
#convert movie rating to class
filter_data = []
discard = []
for each_movie in movie_data:
    rating = each_movie[1]
    c = 0
    if rating == 'Beg or borrow  but do watch' or rating == 'Add to \'must watch\' list'  or rating == 'Owner\'s Pride' :
        c = 1
    elif rating == 'The keen should rent  else TV/online' or rating == 'Watch if you have nothing better to do' or rating == 'Watch when on TV/online':
        c = 2
    elif rating == 'Switch channels if it\'s on cable':
        c = 3
        
    if c !=0:
        filter_data.append([each_movie[0],c,each_movie[2]])
    else:
        discard.append(rating)
 

In [7]:
#after filtering
len(filter_data),len(discard)

(1195, 58)

In [8]:
#Filered Movie Data
filter_data[:2]

[['Rukh', 2, '/movie/rukh-review/'],
 ['Jia Aur Jia', 2, '/movie/jia-aur-jia-review/']]

In [3]:
#np.save('x',filter_data)
filter_data = np.load('x.npy')
len(filter_data)

1195

In [31]:
#Download Full Review Of Each Movie and write it in txt file and store it
path_review='C:\\Users\\Ali\\Repo\\Web Scrapping and Classifier\\reviews' #path to store full movie review

print('download reviews ......................')
for index,each_data in enumerate(filter_data):
    #print(each_data[0])
    
    #Read Page of Full Review
    link = 'https://www.wogma.com/'+each_data[2]
    review_page = urllib.request.urlopen(link).read()
    soup = bs.BeautifulSoup(review_page,'lxml')
    
    #<div> with class 'wogma-review' contain text for our full review
    div = soup.find_all('div',class_='wogma-review')
    review_txt = div[0].get_text()
    store extrated moview review in this path
    complete_path = os.path.join(path_review,str(index)+".txt")
    
    with open(complete_path, "w", encoding="utf-8") as f:
        f.write(review_txt)
print('all review downloaded at ',path_review)

download reviews ......................
all review downloaded at  C:\Users\Ali\Repo\Web Scrapping and Classifier\reviews


# Feature Creation
# We will create corpus from all ~ 1200 review and select top 50-5000 vocabulary 

In [9]:
#list of all reviews
reviews_files = sorted(glob.glob("reviews\*.txt"))

In [10]:
len(reviews_files)

1195

In [11]:
#corpus of all our review
corpus_raw = u""
for file in reviews_files:
     with codecs.open(file,"r","utf-8") as review_file:
            corpus_raw += review_file.read()
print("done")

done


In [12]:
len(corpus_raw)

4022978

In [14]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [13]:
#token of senteces
raw_sentences = tokenizer.tokenize(corpus_raw)

In [14]:
len(raw_sentences)

39749

In [15]:
#remove all special character from our sentence
#and create word token from each sentence
vocab_list = []
for each_sentences in raw_sentences:
    if len(each_sentences)>0:
        clean = re.sub("[^a-zA-Z0-9’]"," ",each_sentences)
        words =  clean.split()
        if len(words)>0:
            for word in words:
                if word != 'adsbygoogle':
                    vocab_list.append(word)

        

In [16]:
len(vocab_list)

704024

In [17]:
w = nltk.FreqDist(vocab_list)

In [18]:
#Top most common word from vocab list
#This word doesnt contrib in our classification 
print(w.most_common(30))

[('the', 38381), ('a', 19241), ('of', 18082), ('to', 16815), ('and', 15501), ('is', 15145), ('in', 11289), ('that', 10269), ('it', 8329), ('film', 7183), ('you', 6609), ('for', 5473), ('with', 5193), ('I', 5163), ('are', 5091), ('The', 5088), ('as', 4447), ('s', 3870), ('have', 3481), ('his', 3406), ('on', 3335), ('t', 3313), ('be', 3293), ('one', 3116), ('not', 3072), ('this', 3070), ('at', 2870), ('by', 2571), ('all', 2560), ('an', 2546)]


In [19]:
#number of unique word
len(w.keys())

30172

In [20]:
#select top 50 to 5000 words from our vocab list
word_features = list(w.keys())[50:5000]

# Now we have 4500 features for each movie ,We have mark 1 if this feature exist 0 if not

In [21]:
def create_features(item):
    review_path = ('reviews\{0}.txt'.format(str(item)))
    
    #read data
    review = u""
    with codecs.open(review_path,"r","utf-8") as review_file:
            review = review_file.read()
            
    #create sentence token
    review_sentence_token = tokenizer.tokenize(review)
    v = []
    #clean sentence and create word token
    for s_token in review_sentence_token:
        clean = re.sub("[^a-zA-Z0-9’]"," ",s_token)
        words =  clean.split()
        for word in words:
            v.append(word)
    v = set(v)
    f = []
    #check if that token exist or not
    for feature in word_features:
        f.append(int(feature in v))
    return f 

# Classification with RandomForest using scikit learn library

In [23]:
X = np.array([create_features(item) for item in range(1195)],dtype=float)
Y = np.array([filter_data[item][1] for item in range(1195)])
#np.random.shuffle(dataset)

In [116]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [26]:
train_x, test_x, train_y, test_y = train_test_split(X,Y,train_size=0.7)

C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [27]:
RanForestClf = RandomForestClassifier()
RanForestClf.fit(train_x,train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [28]:
print(accuracy_score(test_y,RanForestClf.predict(test_x))*100)

59.6100278552


# Approach 2 . 
# Creating Features using wordvector  by Taking average of all words vector of each review

In [6]:
#glove vector training on wikipedia 
#6B tokens, 400K vocab, uncased, 50d, 100d, 200d, & 300d vectors, 822 MB download
glove_vect_300d = 'glove.6B/glove.6B.300d.txt'

In [9]:
glove_300D = pd.read_table(glove_vect_300d, sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

In [10]:
glove_300D.head()

,1,2,3,4,5,6,7,8,9,10,...,291,292,293,294,295,296,297,298,299,300
0,,,,,,,,,,,,,,,,,,,,,
the,0.046560,0.213180,-0.007436,-0.458540,-0.035639,0.236430,-0.288360,0.215210,-0.134860,-1.6413,...,-0.013064,-0.296860,-0.079913,0.195000,0.031549,0.285060,-0.087461,0.009061,-0.209890,0.053913
",",-0.255390,-0.257230,0.131690,-0.042688,0.218170,-0.022702,-0.178540,0.107560,0.058936,-1.3854,...,0.075968,-0.014359,-0.073794,0.221760,0.146520,0.566860,0.053307,-0.232900,-0.122260,0.354990
.,-0.125590,0.013630,0.103060,-0.101230,0.098128,0.136270,-0.107210,0.236970,0.328700,-1.6785,...,0.060148,-0.156190,-0.119490,0.234450,0.081367,0.246180,-0.152420,-0.342240,-0.022394,0.136840
of,-0.076947,-0.021211,0.212710,-0.722320,-0.139880,-0.122340,-0.175210,0.121370,-0.070866,-1.5721,...,-0.366730,-0.386030,0.302900,0.015747,0.340360,0.478410,0.068617,0.183510,-0.291830,-0.046533
to,-0.257560,-0.057132,-0.671900,-0.380820,-0.364210,-0.082155,-0.010955,-0.082047,0.460560,-1.8477,...,-0.012806,-0.597070,0.317340,-0.252670,0.543840,0.063007,-0.049795,-0.160430,0.046744,-0.070621


In [246]:
def create_features(item):
    review_path = ('reviews\{0}.txt'.format(str(item)))
    
    #read data
    review = u""
    with codecs.open(review_path,"r","utf-8") as review_file:
            review = review_file.read()
            
    #create sentence token
    review_sentence_token = tokenizer.tokenize(review)
    v = []
    #clean sentence and create word token
    for s_token in review_sentence_token:
        clean = re.sub("[^a-zA-Z0-9’]"," ",s_token)
        words =  clean.split()
        for word in words:
            v.append(word.lower())
    #v = set(v)
    f = np.zeros(300)
    
    found_vocab = []
    unfound_word_glove= []
    for voc in v:
    #check if vocab is available in vector
    #if yes value of latent features and sum it.
        if voc in glove_300D.index:
            f += glove_300D.loc[voc].as_matrix()
            found_vocab.append(voc)
        else:
            unfound_word_glove.append(voc)
    c = 1
    if len(found_vocab) > 1:
        c = len(found_vocab)
        
        
    #print('{0} word not found ,out of {1}'.format(str(len(unfound_word_glove)),str(len(v))))
    return f/c

In [247]:
X = np.array([create_features(item) for item in range(1195)],dtype=float)
Y = np.array([filter_data[item][1] for item in range(1195)])
#np.random.shuffle(dataset)

In [248]:
train_x, test_x, train_y, test_y = train_test_split(X,Y,train_size=0.7)

C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [272]:
RanForestClf = RandomForestClassifier()
RanForestClf.fit(train_x,train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [273]:
print(accuracy_score(test_y,RanForestClf.predict(test_x))*100)

56.5459610028


In [259]:
from sklearn import linear_model

In [260]:
Lr = linear_model.LogisticRegression(n_jobs=1 ,C=1e5)
Lr.fit(train_x,train_y)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [261]:
print(accuracy_score(test_y,Lr.predict(test_x))*100)

64.3454038997
